<a href="https://colab.research.google.com/github/RhysWangJunfei/nilm/blob/master/nilm_rnn_cde.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import tensorflow as tf
import numpy as np
import io
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split 
from google.colab import files
import matplotlib.pyplot as plt

In [0]:
'''Sliding window function'''
def create_dataset(dataset, look_back=1):
    dataX = []
    for i in range(len(dataset)-look_back+1):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
    return np.array(dataX)

In [4]:
uploaded = files.upload()
'''Load data'''
WHE_data = pd.read_csv(io.BytesIO(uploaded['Electricity_WHE.csv']))['P']
#WHE_data = pd.read_csv('Electricity_WHE.csv')['P']
'''
CDE_data = pd.read_csv(base_dir+'Electricity_CDE.csv')['P']
CWE_data = pd.read_csv(base_dir+'Electricity_CWE.csv')['P']
DWE_data = pd.read_csv(base_dir+'Electricity_DWE.csv')['P']
FRE_data = pd.read_csv(base_dir+'Electricity_FRE.csv')['P']
HPE_data = pd.read_csv(base_dir+'Electricity_HPE.csv')['P']
HTE_data = pd.read_csv(base_dir+'Electricity_HTE.csv')['P']
'''

Saving Electricity_WHE.csv to Electricity_WHE.csv


"\nCDE_data = pd.read_csv(base_dir+'Electricity_CDE.csv')['P']\nCWE_data = pd.read_csv(base_dir+'Electricity_CWE.csv')['P']\nDWE_data = pd.read_csv(base_dir+'Electricity_DWE.csv')['P']\nFRE_data = pd.read_csv(base_dir+'Electricity_FRE.csv')['P']\nHPE_data = pd.read_csv(base_dir+'Electricity_HPE.csv')['P']\nHTE_data = pd.read_csv(base_dir+'Electricity_HTE.csv')['P']\n"

In [0]:
#uploaded = files.upload()
#CDE_data = pd.read_csv(io.BytesIO(uploaded['Electricity_CDE.csv']))['P']
CDE_data = pd.read_csv('Electricity_CDE.csv')['P']

In [0]:
window_size=60

dataX_raw = create_dataset(WHE_data.as_matrix(), window_size)

#0,1-350,4401-5300,others

cde_Y_raw = CDE_data[window_size-1:].values.reshape([CDE_data.shape[0]-window_size+1,1])
dataX = np.concatenate([dataX_raw[0:472500,:],dataX_raw[475500:,:]],axis=0)
cde_Y = np.concatenate([cde_Y_raw[0:472500,:],cde_Y_raw[475500:,:]],axis=0)
categorized_cde_Y = np.ones(cde_Y.shape)*3
categorized_cde_Y[[np.where(cde_Y==0)[0]],:]=0
categorized_cde_Y[[np.where((cde_Y>0)&(cde_Y<=350))[0]],:]=1
categorized_cde_Y[[np.where((cde_Y>4400)&(cde_Y<=5300))[0]],:]=2

#cdeY=categorized_cde_Y[categorized_cde_Y>0].reshape(-1,1)
#cdeX=dataX[[np.where(categorized_cde_Y>0)[0]],:][0]


encoder = OneHotEncoder(handle_unknown='ignore',sparse=False)
cdeY_1hot = encoder.fit_transform(categorized_cde_Y)

X_train, X_test, y_train, y_test = train_test_split(dataX, cdeY_1hot, test_size=0.1, shuffle=True)
scaler = MinMaxScaler()
scaler.fit(X_train.astype(float))
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [0]:
from imblearn.under_sampling import RandomUnderSampler
tl = RandomUnderSampler(sampling_strategy={0:9000,1:9000,2:9000})
X_train, y_train = tl.fit_resample(X_train, y_train)
whole_train = np.concatenate([X_train,y_train],axis=1)
#from imblearn.under_sampling import EditedNearestNeighbours
#enn = EditedNearestNeighbours()
#X_train, y_train = enn.fit_resample(X_train, y_train)

#from imblearn.under_sampling import NeighbourhoodCleaningRule
#ncr = NeighbourhoodCleaningRule()
#X_train, y_train = ncr.fit_resample(X_train, y_train)

In [0]:
'''Hyper parameters for deep learning'''
# Hyper Parameters
LR = 0.001               # learning rate
#cfg_list = nf.model_configs()
#error_list = []

#hyperparameters
batch_size=1000
unit_num=128

In [0]:
'''RNN Model Definition'''
tf.reset_default_graph()
''''''
#define inputs
tf_x = tf.placeholder(tf.float32, [None, window_size,1],name='x')
tf_y = tf.placeholder(tf.int32, [None, 4],name='y')

lstm_cell =tf.contrib.rnn.BasicLSTMCell(num_units=unit_num,name='lstm_cell')
outputs, (h_c, h_n) = tf.nn.dynamic_rnn(
    lstm_cell,                   # cell you have chosen
    tf_x,                      # input
    initial_state=None,         # the initial hidden state
    dtype=tf.float32,           # must given if set initial_state = None
    time_major=False,           # False: (batch, time step, input); True: (time step, batch, input)
)
l1 = tf.layers.dense(outputs[:, -1, :],64,activation=tf.nn.leaky_relu,name='l1')
l2 = tf.layers.dense(l1,32,activation=tf.nn.leaky_relu,name='l2')
l3 = tf.layers.dense(l2,16,activation=tf.nn.leaky_relu,name='l3')
pred = tf.layers.dense(l3,4,activation=tf.nn.relu,name='pred')

with tf.name_scope('loss'):
    cross_entropy =  tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_y, logits=pred) 
    loss = tf.reduce_mean(cross_entropy)
    tf.summary.scalar("loss",tensor=loss)

train_op = tf.train.AdamOptimizer(LR).minimize(loss)

accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(tf_y, axis=1), tf.argmax(pred, axis=1)), tf.float32))

init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer()) 


In [0]:
sess = tf.Session()
#sess.run(init_op)
#latest_checkpoint = tf.train.latest_checkpoint('my_net_1')
#meta_path = '%s.meta' % latest_checkpoint
#saver = tf.train.import_meta_graph(meta_path)
saver.restore(sess, 'my_net_3/save_net_rnn.ckpt')
#saver.restore(sess, "my_net_1/save_net_rnn.ckpt")
#graph = tf.get_default_graph()
#tf_x = graph.get_tensor_by_name("x:0")
#tf_y = graph.get_tensor_by_name("y:0")

#saver = tf.train.Saver()
#batch_num = len(X_train)//batch_size
flag = False
for j in range(0,100):
    print('###iteration: '+str(j)+'###')
    batch_index = np.random.choice(20000,batch_size)
    batch_train = whole_train[batch_index,:]
    batch_X = batch_train[:,:-4].reshape([batch_size,window_size,1])
    batch_y = batch_train[:,-4:]
    sess.run(train_op,{tf_x:batch_X , tf_y:batch_y})
    cost_ = sess.run(loss,{tf_x:batch_X, tf_y:batch_y})
    acc_train = sess.run(accuracy,{tf_x:batch_X, tf_y:batch_y})
    acc_test = sess.run(accuracy,feed_dict={tf_x: X_test.reshape([X_test.shape[0],window_size,1]), tf_y:y_test})
    print('train loss= %.4f' % cost_+', Acc=%.2f'% acc_train)
    print('Test Acc=%.2f'% acc_test)
    
    pre = sess.run(pred,feed_dict={tf_x: batch_X, tf_y:batch_y})
    y_lables_argmax = tf.argmax(tf_y,axis=1)  
    y_pred_argmax = tf.argmax(pre,axis=1)
    confusion = tf.confusion_matrix(labels=y_lables_argmax, predictions=y_pred_argmax, num_classes=4)
    #print('Confusion Matrix: \n\n', tf.Tensor.eval(confusion,feed_dict=None))
    print(confusion.eval(session=sess,feed_dict={tf_x: batch_X, tf_y:batch_y}))
    if acc_train>=0.99:
      print(j)
      flag = True
      break
    if(flag==True):
        print(flag)
        break
pre = sess.run(pred,feed_dict={tf_x: X_test.reshape([X_test.shape[0],window_size,1]), tf_y: y_test})
y_lables_argmax = np.argmax(y_test,1)
y_pred_argmax = np.argmax(pre,1)
confusion = tf.confusion_matrix(labels=y_lables_argmax, predictions=y_pred_argmax, num_classes=4)
#print('Confusion Matrix: \n\n', tf.Tensor.eval(confusion,feed_dict=None))
print(confusion.eval(session=sess))
save_path = saver.save(sess, "my_net_3/save_net_rnn.ckpt")
sess.close()


INFO:tensorflow:Restoring parameters from my_net_3/save_net_rnn.ckpt
###iteration: 0###
train loss= 0.6499, Acc=0.70
Test Acc=0.93
[[393  32   0   0]
 [166 307   0   0]
 [ 14  87   1   0]
 [  0   0   0   0]]
###iteration: 1###
train loss= 0.6046, Acc=0.73
Test Acc=0.89
[[424  48   0   0]
 [133 300   1   0]
 [ 13  77   4   0]
 [  0   0   0   0]]
###iteration: 2###
train loss= 0.6216, Acc=0.73
Test Acc=0.86
[[403  57   0   0]
 [110 316   3   0]
 [ 10  91  10   0]
 [  0   0   0   0]]
###iteration: 3###
train loss= 0.6400, Acc=0.71
Test Acc=0.88
[[362  56   0   0]
 [130 327   4   0]
 [ 16  80  25   0]
 [  0   0   0   0]]
###iteration: 4###
train loss= 0.6094, Acc=0.74
Test Acc=0.92
[[437  36   0   0]
 [151 280   8   0]
 [ 12  57  19   0]
 [  0   0   0   0]]
###iteration: 5###
train loss= 0.5937, Acc=0.75
Test Acc=0.93
[[423  25   0   0]
 [139 312   4   0]
 [ 18  68  11   0]
 [  0   0   0   0]]
###iteration: 6###


In [0]:
#!mkdir my_net
#mv checkpoint save_net_rnn.ckpt.data-00000-of-00001 save_net_rnn.ckpt.index save_net_rnn.ckpt.meta my_net
rm -r my_net_3